In [1]:
import numpy as np
from sklearn.datasets import fetch_20newsgroups
import matplotlib.pyplot as plt

%matplotlib inline
np.set_printoptions(suppress=True)

In [2]:
categories = ['alt.atheism', 'talk.religion.misc', 'comp.graphics', 'sci.space']
remove = ('headers', 'footers', 'quotes')
newsgroups_train = fetch_20newsgroups(subset='train', categories=categories, remove=remove)
newsgroups_test = fetch_20newsgroups(subset='test', categories=categories, remove=remove)

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [4]:
vectorizer_tfidf = TfidfVectorizer(stop_words='english')
vectors_tfidf = vectorizer_tfidf.fit_transform(newsgroups_train.data).todense() # (documents, vocab)
vectors_tfidf.shape

(2034, 26576)

In [5]:
num_top_words=8
vocab = np.array(vectorizer_tfidf.get_feature_names())

def show_topics(a):
    top_words = lambda t: [vocab[i] for i in np.argsort(t)[:-num_top_words-1:-1]]
    topic_words = ([top_words(t) for t in a])
    return [' '.join(t) for t in topic_words]

### NMF from sklearn 

In [6]:
from sklearn import decomposition

d = 5 # num topics
clf = decomposition.NMF(n_components=d, random_state=1)

In [7]:
W1 = clf.fit_transform(vectors_tfidf)
H1 = clf.components_

In [8]:
show_topics(H1)

['people don think just like objective say morality',
 'graphics thanks files image file program windows know',
 'space nasa launch shuttle orbit moon lunar earth',
 'ico bobbe tek beauchaine bronx manhattan sank queens',
 'god jesus bible believe christian atheism does belief']

### NMF from scratch using SGD

In stochastic gradient descent (SGD), we evaluate our loss function on just a sample of our data (sometimes called a mini-batch). We would get different loss values on different samples of the data, so this is why it is stochastic. It turns out that this is still an effective way to optimize, and it's much more efficient!


Applying SGD to NMF

Goal: Decompose $V\;(m \times n)$ into $$V \approx WH$$ where $W\;(m \times d)$ and $H\;(d \times n)$, $W,\;H\; \geq \;0$, and we've minimized the Frobenius norm of $V-WH$.

Approach: We will pick random positive $W$ & $H$, and then use SGD to optimize.

#### using numpy

In [9]:
def grads(M, W, H):
    R = W@H-M
    return R@H.T + penalty(W, mu)*lam, W.T@R + penalty(H, mu)*lam # dW, dH

In [10]:
def penalty(M, mu):
    return np.where(M>=mu, 0, np.min(M - mu, 0))

In [11]:
def upd(M, W, H, lr):
    dW,dH = grads(M,W,H)
    W -= lr*dW; H -= lr*dH

In [12]:
def report(M,W,H): 
    print(np.linalg.norm(M-W@H), W.min(), H.min(), (W<0).sum(), (H<0).sum())

In [13]:
m, n = vectors_tfidf.shape

W = np.abs(np.random.normal(scale=0.01, size=(m,d)))
H = np.abs(np.random.normal(scale=0.01, size=(d,n)))

In [14]:
mu = 1e-6
lam = 1e3
lr = 1e-2

In [15]:
%%time
for i in range(50): 
    upd(vectors_tfidf,W,H,lr)
    if i % 10 == 0: report(vectors_tfidf,W,H)

44.4179745263 -0.000848008936649 -7.1156988209e-05 137 306
44.3774945487 -0.000327947903727 -5.85181157935e-05 46 450
44.3487344257 -0.000235325707557 -8.46750926792e-05 33 925
44.317056816 -0.000169573014055 -8.40447187289e-05 36 1422
44.2820416704 -0.000105532968983 -0.000102598904842 38 2117
CPU times: user 1min 45s, sys: 19.4 s, total: 2min 4s
Wall time: 38.6 s


In [16]:
show_topics(H)

['don god space know think people just like',
 'god space just people like don think does',
 'space people don like think does god just',
 'god don know space people just think does',
 'god don space people think just time like']

This method is very slow to train!

#### using PyTorch

In [17]:
import torch
from scipy import sparse

In [18]:
def grads_t(M, W, H):
    R = W.mm(H)-M
    return (R.mm(H.t()) + penalty_t(W, mu)*lam, 
        W.t().mm(R) + penalty_t(H, mu)*lam) # dW, dH

def penalty_t(M, mu):
    return (M<mu).type(torch.FloatTensor)*torch.clamp(M - mu, max=0.)

def upd_t(M, W, H, lr):
    dW,dH = grads_t(M,W,H)
    W.sub_(lr*dW); H.sub_(lr*dH)

def report_t(M,W,H): 
    print((M-W.mm(H)).norm(2), W.min(), H.min(), (W<0).sum(), (H<0).sum())

In [19]:
t_vectors = torch.Tensor(vectors_tfidf.astype(np.float32))

t_W = torch.FloatTensor(m,d).normal_(std=0.01).abs_()
t_H = torch.FloatTensor(d,n).normal_(std=0.01).abs_()

In [20]:
%%time
for i in range(50): 
    upd_t(t_vectors,t_W,t_H,lr)
    if i % 10 == 0: 
        report_t(t_vectors,t_W,t_H)
        lr *= 0.9

44.41695271374714 -0.0009815343655645847 -7.52935666241683e-05 138 250
44.380284272501655 -0.0004332782991696149 -4.4321739551378414e-05 127 432
44.35766005798296 -0.00019702206191141158 -5.1875977078452706e-05 123 872
44.33610793274646 -0.00014351926802191883 -5.264638821245171e-05 112 1368
44.31470034494246 -0.0001014827357721515 -6.394876254489645e-05 131 2019
CPU times: user 54.8 s, sys: 8.11 s, total: 1min 2s
Wall time: 16.3 s


In [21]:
show_topics(t_H.numpy())

['god space people know think just don like',
 'space god don think people just like know',
 'don space people just like god does know',
 'don just people space think god know time',
 'space don god know like people just does']

#### using PyTorch and Autograd

In [22]:
from torch.autograd import Variable

In [23]:
def penalty(A):
    return torch.pow((A<0).type(torch.FloatTensor)*torch.clamp(A, max=0.), 2)

def penalize(): 
    return penalty(pW).mean() + penalty(pH).mean()

def loss(): 
    return (M-pW.mm(pH)).norm(2) + penalize()*lam

def report():
    W,H = pW.data, pH.data
    print((M-pW.mm(pH)).norm(2).data[0], W.min(), H.min(), (W<0).sum(), (H<0).sum())

In [24]:
M = Variable(t_vectors)

pW = Variable(torch.FloatTensor(m,d), requires_grad=True)
pH = Variable(torch.FloatTensor(d,n), requires_grad=True)
pW.data.normal_(std=0.01).abs_()
pH.data.normal_(std=0.01).abs_();

In [25]:
opt = torch.optim.Adam([pW,pH], lr=1e-3, betas=(0.9,0.9))

In [26]:
%%time
for i in range(50): 
    opt.zero_grad()
    l = loss()
    l.backward()
    opt.step()
    if i % 10 == 0: 
        report()
        lr *= 0.9     # learning rate annealling

44.40464782714844 -0.0009986847871914506 -0.0009995140135288239 499 8492
44.29405975341797 -0.0060671851970255375 -0.010416567325592041 659 54102
44.22111892700195 -0.007334951777011156 -0.01876099593937397 499 54590
44.1679801940918 -0.00876869261264801 -0.024152137339115143 1068 60384
44.13126754760742 -0.01158764399588108 -0.031652577221393585 1525 59600
CPU times: user 1min 45s, sys: 19.7 s, total: 2min 5s
Wall time: 41.5 s


In [27]:
show_topics(pH.data.numpy())

['way post example know using thank things exist',
 'religion make software god years time didn real',
 'moon morality data louis years like called idea',
 'christian color povray file rtrace post want help',
 'sure actually line using atheism mac bible religion']